# Aprendizagem supervisionada: classificação

Este projeto tem como objetivo desenvolver um aplicativo de machine learning para prever a tendência de uma pessoa desenvolver algum tipo de 
doença cardíaca com base em alguns fatores clínicos e laboratoriais.

Os dados foram extraídos do Kaggle: https://www.kaggle.com/fedesoriano/heart-failure-prediction/version/1

## Pré-processamento

In [1]:
import numpy as np
import pandas as pd

In [2]:
# apesar de ter sido salvo como utf-8, neste caso (não é recomendável), poderia usar iso-8859-1 que também funciona
# df = pd.read_csv('../datasets/heart_tratado.csv', sep=';', encoding='iso-8859-1')
df = pd.read_csv('../datasets/heart_tratado.csv', sep=';', encoding='utf-8')

In [3]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289.0,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180.0,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283.0,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214.0,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195.0,0,Normal,122,N,0.0,Up,0


## Transformando variáveis categóricas nominais em variáveis categóricas ordinais

Onde a categoria for um nome, está será transformada em número, por exemplo: Sex, M ou F, para 0 e 1, ou 1 e 2...

In [7]:
df2 = pd.DataFrame.copy(df)

In [43]:
df2['Sex'].replace({'M': 0, 'F': 1}, inplace=True)
df2['ChestPainType'].replace({'TA': 0, 'ATA': 1, 'NAP': 2, 'ASY': 3}, inplace=True)
df2['RestingECG'].replace({'Normal': 0, 'ST': 1, 'LVH': 2}, inplace=True)
df2['ExerciseAngina'].replace({'N': 0, 'Y': 1}, inplace=True)
df2['ST_Slope'].replace({'Up': 0, 'Flat': 1, 'Down': 2}, inplace=True)

C:\Users\rafae\AppData\Local\Temp\ipykernel_15328\3930933072.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2['RestingECG'].replace({'Normal': 0, 'ST': 1, 'LVH': 2}, inplace=True)
C:\Users\rafae\AppData\Local\Temp\ipykernel_15328\3930933072.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  

In [44]:
df2.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,0,1,140,289.0,0,0,172,0,0.0,0,0
1,49,1,2,160,180.0,0,0,156,0,1.0,1,1
2,37,0,1,130,283.0,0,1,98,0,0.0,0,0
3,48,1,3,138,214.0,0,0,108,1,1.5,1,1
4,54,0,2,150,195.0,0,0,122,0,0.0,0,0


## Legenda

- Age = idade
- Sex = sexo
- Chest Pain Type = tipo de dor no peito (TA = angina típica, ATA = angina atípica, NAP = dor não anginosa, ASY = assintomático)
- Resting BP = pressão sanguínea em repouso (mmHg)
- Cholesterol = colesterol sérico (mg/dl)
- Fasting BS = açúcar no sangue em jejum (mg/dl), < 120 = não-diabético, >= 120 = diabético
- Resting ECG = eletrocardiograma em repouso (Normal, ST = Anormalidade da onda ST-T, LVH = Hipertrofia ventricular esquerda)
- Max HR = frequência cardíaca máxima
- Exercise Angina = Angina induzida por exercício
- Old Peak = depressão de ST induzida por exercício em relação ao repouso
- ST_Slope = inclinação do segmento ST (Up, Flat, Down)
- Heart Disease = Doença cardíaca (possui / não possui)

## Atributos previsores e alvo

In [ ]:
# a função iloc(), busca considerando um índice
# Neste caso, será criado uma matrix com os valores, até a 10a coluna
previsores = df2.iloc[:, 0:11].values
# warning
# We recommend using DataFrame.to_numpy instead.
# Only the values in the DataFrame will be returned, the axes labels will be removed.
# previsores = df2.to_numpy() # Assim fica com uma coluna a mais -> previsores.shape = (917, 12)
# Corrigindo:
# previsores = df2.to_numpy()[:, 0:11]


In [46]:
previsores.shape

(917, 11)

In [39]:
print(type(previsores))
print(previsores)

<class 'numpy.ndarray'>
[[40 0 1 ... 0 0.0 0]
 [49 1 2 ... 0 1.0 1]
 [37 0 1 ... 0 0.0 0]
 ...
 [57 0 3 ... 1 1.2 1]
 [57 1 1 ... 0 0.0 1]
 [38 0 2 ... 0 0.0 0]]


In [ ]:
# só irá pegar a coluna 11, gerando um array com seus valores
alvo = df2.iloc[:, 11].values

In [14]:
alvo.shape

(917,)

In [28]:
alvo

array([0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,

In [17]:
df2.describe()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
count,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.00000,917.000000
mean,53.509269,0.210469,2.251908,132.540894,244.635389,0.233370,136.789531,0.404580,0.886696,0.63795,0.552890
std,9.437636,0.407864,0.931502,17.999749,53.347125,0.423206,25.467129,0.491078,1.066960,0.60727,0.497466
min,28.000000,0.000000,0.000000,80.000000,85.000000,0.000000,60.000000,0.000000,-2.600000,0.00000,0.000000
25%,47.000000,0.000000,2.000000,120.000000,214.000000,0.000000,120.000000,0.000000,0.000000,0.00000,0.000000
50%,54.000000,0.000000,3.000000,130.000000,244.635389,0.000000,138.000000,0.000000,0.600000,1.00000,1.000000
75%,60.000000,0.000000,3.000000,140.000000,267.000000,0.000000,156.000000,1.000000,1.500000,1.00000,1.000000
max,77.000000,1.000000,3.000000,200.000000,603.000000,1.000000,202.000000,1.000000,6.200000,2.00000,1.000000


Padronização (usa média e o desvio padrão como referência)
Normalização (usa os valores de máximo e mínimo como referência)

Será usada a padronização

In [18]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# verificando se há valores de string, porque se tiver, vai dar erro no fit_transform()
# has_string = np.any([isinstance(item, str) for item in previsores.flatten()])
# print('Contém strings?', has_string) # true

# string_values = set([item for item in previsores.flatten() if isinstance(item, str)])
# recommended way by sonar
# string_values = {item for item in previsores.flatten() if isinstance(item, str)}
# print("Valores string encontrados:", string_values)

# escalonamento dos previsores
scaler = StandardScaler()
previsores_esc = scaler.fit_transform(previsores) # ValueError: could not convert string to float: 'Normal'

In [49]:
previsores_esc

array([[-1.43220634, -0.51630861, -1.34470119, ..., -0.82431012,
        -0.83150225, -1.05109458],
       [-0.47805725,  1.9368261 , -0.27058012, ..., -0.82431012,
         0.10625149,  0.59651863],
       [-1.75025603, -0.51630861, -1.34470119, ..., -0.82431012,
        -0.83150225, -1.05109458],
       ...,
       [ 0.37007527, -0.51630861,  0.80354095, ...,  1.21313565,
         0.29380223,  0.59651863],
       [ 0.37007527,  1.9368261 , -1.34470119, ..., -0.82431012,
        -0.83150225,  0.59651863],
       [-1.64423947, -0.51630861, -0.27058012, ..., -0.82431012,
        -0.83150225, -1.05109458]], shape=(917, 11))

### Codificação de variáveis categóricas

#### LabelEncoder: transformação de variáveis categóricas em numéricas

In [50]:
from sklearn.preprocessing import LabelEncoder

In [53]:
# no caso estamos usando o dataset original tratado com os dados nominais
previsores2 = df.to_numpy()[:, 0:11]   # outra forma de fazer df2.iloc[:, 0:11].values

In [ ]:
previsores2

array([[40, 'M', 'ATA', ..., 'N', 0.0, 'Up'],
       [49, 'F', 'NAP', ..., 'N', 1.0, 'Flat'],
       [37, 'M', 'ATA', ..., 'N', 0.0, 'Up'],
       ...,
       [57, 'M', 'ASY', ..., 'Y', 1.2, 'Flat'],
       [57, 'F', 'ATA', ..., 'N', 0.0, 'Flat'],
       [38, 'M', 'NAP', ..., 'N', 0.0, 'Up']],
      shape=(917, 11), dtype=object)

#### Aplicando LabelEncoder em todas as colunas que contém strings

In [ ]:
str_cols = [col for col in df.columns if df[col].apply(type).eq(str).any()]
str_cols

['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

In [61]:
cols_with_index = {col: df.columns.get_loc(col) for col in str_cols}
cols_with_index

{'Sex': 1,
 'ChestPainType': 2,
 'RestingECG': 6,
 'ExerciseAngina': 8,
 'ST_Slope': 10}

In [69]:
print(cols_with_index['Sex'])

1


In [70]:
encoder = LabelEncoder()

for col in str_cols:
    col_index = cols_with_index[col]
    previsores2[:, col_index] = encoder.fit_transform(previsores2[:, col_index])

In [71]:
previsores2

array([[40, 1, 1, ..., 0, 0.0, 2],
       [49, 0, 2, ..., 0, 1.0, 1],
       [37, 1, 1, ..., 0, 0.0, 2],
       ...,
       [57, 1, 0, ..., 1, 1.2, 1],
       [57, 0, 1, ..., 0, 0.0, 1],
       [38, 1, 2, ..., 0, 0.0, 2]], shape=(917, 11), dtype=object)

### OneHotEncoder: criação de variáveis dummy (fictícias)

Cuidado com a multicolinearidade (variáveis altamente correlacionadas entre si).
<p>
Exemplo: "Você faz atividade física?"<br/>
A = 0 ==> não<br/>
B = 1 ==> sim, 1 ou 2 dias por semana<br/>
C = 2 ==> sim, 3 ou 4 dias por semana<br/>
D = 3 ==> sim, pelo menos 5 dias por semana
</p>

In [72]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [73]:
ColumnTransformer?

Init signature:
ColumnTransformer(
    transformers,
    *,
    remainder='drop',
    sparse_threshold=0.3,
    n_jobs=None,
    transformer_weights=None,
    verbose=False,
    verbose_feature_names_out=True,
    force_int_remainder_cols='deprecated',
)
Docstring:     
Applies transformers to columns of an array or pandas DataFrame.

This estimator allows different columns or column subsets of the input
to be transformed separately and the features generated by each transformer
will be concatenated to form a single feature space.
This is useful for heterogeneous or columnar data, to combine several
feature extraction mechanisms or transformations into a single transformer.

Read more in the :ref:`User Guide <column_transformer>`.

.. versionadded:: 0.20

Parameters
----------
transformers : list of tuples
    List of (name, transformer, columns) tuples specifying the
    transformer objects to be applied to subsets of the data.

    name : str
        Like in Pipeline and FeatureUnion

Parâmetros ColumnTransformer:
- name = nome da transformação
- transformer = tipo de estimador (OneHotEncoder)
- columns = colunas que serão transformadas
- remainder = o que acontecerá com o restante das colunas não relacionadas: 1) drop = exclui as outras, 2) passthrough = mantém (default: drop)
- sparse_threshould = parâmetro de clasificacao de matrizes esparsas. default = 0.3
- n_job = numero de trabalhos executados em paralelo. default = nenhum
- transformer_weights = definicao de pesos aos transformadores
- verbose = exibicao da execucao na tela. default false